In [29]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import OneHotEncoder
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [30]:
data = pd.read_csv("traffic_data.csv")
features = data[['timestamp', 'intensity', 'velocity', 'lat', 'long', 'tweet count', 'weather','target']]
features['timestamp'] = pd.to_datetime(features['timestamp'])
features['month'] = features['timestamp'].dt.month
features['day'] = features['timestamp'].dt.day
features['hour'] = features['timestamp'].dt.hour
features = features.drop('timestamp', axis=1)
lat = features['lat']
long = features['long']

C:\Users\Anup\AppData\Local\Temp\ipykernel_24048\1525057717.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['timestamp'] = pd.to_datetime(features['timestamp'])
C:\Users\Anup\AppData\Local\Temp\ipykernel_24048\1525057717.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['month'] = features['timestamp'].dt.month


In [31]:
features.head()

,intensity,velocity,lat,long,tweet count,weather,target,month,day,hour
0,5520,80,-3.674076,40.48064,9,partly-cloudy-night,1,5,11,9
1,5640,80,-3.674076,40.48064,32,partly-cloudy-night,1,5,11,9
2,5880,78,-3.674076,40.48064,51,partly-cloudy-night,1,5,11,9
3,5580,80,-3.674076,40.48064,32,partly-cloudy-night,1,5,11,9
4,4980,78,-3.674076,40.48064,55,partly-cloudy-night,1,5,11,10


In [32]:
unique_values_1 = features['weather'].unique()
print(unique_values_1)

['partly-cloudy-night' 'clear-night' 'clear-day' 'partly-cloudy-day'
 'cloudy' 'rain' 'fog']


In [33]:
mapping = {value: i for i, value in enumerate(unique_values_1)}

In [34]:
features['weather'] = features['weather'].replace(mapping)

In [35]:
features.head()

,intensity,velocity,lat,long,tweet count,weather,target,month,day,hour
0,5520,80,-3.674076,40.48064,9,0,1,5,11,9
1,5640,80,-3.674076,40.48064,32,0,1,5,11,9
2,5880,78,-3.674076,40.48064,51,0,1,5,11,9
3,5580,80,-3.674076,40.48064,32,0,1,5,11,9
4,4980,78,-3.674076,40.48064,55,0,1,5,11,10


In [37]:
target = features['target']
features = features.drop(['target','month'],axis=1)
features

,intensity,velocity,lat,long,tweet count,weather,day,hour
0,5520,80,-3.674076,40.48064,9,0,11,9
1,5640,80,-3.674076,40.48064,32,0,11,9
2,5880,78,-3.674076,40.48064,51,0,11,9
3,5580,80,-3.674076,40.48064,32,0,11,9
4,4980,78,-3.674076,40.48064,55,0,11,10
...,...,...,...,...,...,...,...,...
6017,1140,70,-3.674579,40.47598,60,3,25,18
6018,1080,72,-3.674579,40.47598,53,3,25,18
6019,1260,74,-3.674579,40.47598,53,3,25,19
6020,1320,73,-3.674579,40.47598,53,3,25,19


In [38]:
features.head()

,intensity,velocity,lat,long,tweet count,weather,day,hour
0,5520,80,-3.674076,40.48064,9,0,11,9
1,5640,80,-3.674076,40.48064,32,0,11,9
2,5880,78,-3.674076,40.48064,51,0,11,9
3,5580,80,-3.674076,40.48064,32,0,11,9
4,4980,78,-3.674076,40.48064,55,0,11,10


In [39]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import LSTM, Dense
# import numpy as np

# Preprocess the data
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
print(y_train[:5])
# Reshape the input data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM model
model = Sequential()
# model.add(LSTM(128, input_shape=(1, X_train.shape[2])))
# model.add(Dense(1, activation='sigmoid'))

#modified :
model.add(LSTM(64, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32)



1483    0
4703    1
5766    0
2788    0
4139    1
Name: target, dtype: int64
Epoch 1/100
151/151 [==============================] - 6s 4ms/step - loss: 0.6321 - accuracy: 0.6591
Epoch 2/100
151/151 [==============================] - 1s 4ms/step - loss: 0.5243 - accuracy: 0.7168
Epoch 3/100
151/151 [==============================] - 1s 5ms/step - loss: 0.4806 - accuracy: 0.7565
Epoch 4/100
151/151 [==============================] - 1s 5ms/step - loss: 0.4527 - accuracy: 0.7845
Epoch 5/100
151/151 [==============================] - 1s 5ms/step - loss: 0.4346 - accuracy: 0.7893
Epoch 6/100
151/151 [==============================] - 1s 5ms/step - loss: 0.4295 - accuracy: 0.7939
Epoch 7/100
151/151 [==============================] - 1s 5ms/step - loss: 0.4171 - accuracy: 0.8028
Epoch 8/100
151/151 [==============================] - 1s 5ms/step - loss: 0.4131 - accuracy: 0.8117
Epoch 9/100
151/151 [==============================] - 1s 5ms/step - loss: 0.4059 - accuracy: 0.8169
Epoch 10/100
1

151/151 [==============================] - 1s 5ms/step - loss: 0.3308 - accuracy: 0.8628
Epoch 81/100
151/151 [==============================] - 1s 5ms/step - loss: 0.3314 - accuracy: 0.8599
Epoch 82/100
151/151 [==============================] - 1s 5ms/step - loss: 0.3278 - accuracy: 0.8632
Epoch 83/100
151/151 [==============================] - 1s 5ms/step - loss: 0.3323 - accuracy: 0.8588
Epoch 84/100
151/151 [==============================] - 1s 4ms/step - loss: 0.3312 - accuracy: 0.8607
Epoch 85/100
151/151 [==============================] - 1s 4ms/step - loss: 0.3292 - accuracy: 0.8638
Epoch 86/100
151/151 [==============================] - 1s 4ms/step - loss: 0.3254 - accuracy: 0.8655
Epoch 87/100
151/151 [==============================] - 1s 4ms/step - loss: 0.3243 - accuracy: 0.8657
Epoch 88/100
151/151 [==============================] - 1s 5ms/step - loss: 0.3246 - accuracy: 0.8669
Epoch 89/100
151/151 [==============================] - 1s 4ms/step - loss: 0.3260 - accuracy: 

In [43]:

# Assuming you have a single data point to predict
data_point = np.array([[5520,80,-3.674076,	40.480649,21,0,11,9]])  # Replace ... with your data

# Scale and reshape the data point
# scaled_data_point = scaler.transform(data_point)
# reshaped_data_point = np.reshape(scaled_data_point, (1, 1, scaled_data_point.shape[1]))

# # Predict using the trained model
# prediction = model.predict(reshaped_data_point)

scaled_data_point = scaler.transform(data_point)
reshaped_data_point = np.reshape(scaled_data_point, (1, 1, scaled_data_point.shape[1]))

# Predict using the trained model
prediction = model.predict(reshaped_data_point)
# Convert the prediction to congestion or not
if prediction > 0.5:
    congestion = f"Congestion chances: {prediction}"
else:
    congestion = f"No Congestion {prediction}"

# Print the prediction
print("Prediction:", congestion)

1/1 [==============================] - 0s 20ms/step
Prediction: Congestion chances: [[0.96141136]]


C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [42]:
import pickle
# with open('fmodel.pkl','wb') as f:
#     pickle.dump(model,f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Reshape the input data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))
#modified

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
model.add(LSTM(64, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(patience=5, restore_best_weights=True)


history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


huhuhuh
Epoch 1/50
121/121 [==============================] - 4s 19ms/step - loss: 0.6500 - accuracy: 0.6520 - val_loss: 0.6347 - val_accuracy: 0.6805
Epoch 2/50
121/121 [==============================] - 2s 13ms/step - loss: 0.6437 - accuracy: 0.6520 - val_loss: 0.6197 - val_accuracy: 0.6805
Epoch 3/50
121/121 [==============================] - 2s 13ms/step - loss: 0.6361 - accuracy: 0.6520 - val_loss: 0.6059 - val_accuracy: 0.6805
Epoch 4/50
121/121 [==============================] - 2s 16ms/step - loss: 0.6258 - accuracy: 0.6512 - val_loss: 0.5895 - val_accuracy: 0.6857
Epoch 5/50
121/121 [==============================] - 2s 14ms/step - loss: 0.5530 - accuracy: 0.7028 - val_loss: 0.5474 - val_accuracy: 0.7199
Epoch 6/50
121/121 [==============================] - 2s 12ms/step - loss: 0.5335 - accuracy: 0.7137 - val_loss: 0.4883 - val_accuracy: 0.7490
Epoch 7/50
121/121 [==============================] - 2s 13ms/step - loss: 0.5097 - accuracy: 0.7311 - val_loss: 0.4725 - val_accuracy

TypeError: in user code:

    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Anup\AppData

In [87]:
print("uhuh")

In [27]:
# with open('scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

In [45]:

# Scale and reshape the data point
with open('Api/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

scaled_data_point = scaler.transform(data_point)
reshaped_data_point = np.reshape(scaled_data_point, (1, 1, scaled_data_point.shape[1]))

# Predict using the trained model
prediction = model.predict(reshaped_data_point)

# a = model.predict(ll)
print(prediction)

1/1 [==============================] - 0s 25ms/step
[[0.96141136]]


C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [55]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))


# the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))  # layer for regularization
model.add(LSTM(128))
model.add(Dropout(0.2))  # layer for regularization
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# validation mechanism
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train 
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

#testing
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/50
121/121 [==============================] - 7s 29ms/step - loss: 0.6559 - accuracy: 0.6481 - val_loss: 0.6359 - val_accuracy: 0.6805
Epoch 2/50
121/121 [==============================] - 3s 26ms/step - loss: 0.6476 - accuracy: 0.6520 - val_loss: 0.6257 - val_accuracy: 0.6805
Epoch 3/50
121/121 [==============================] - 3s 26ms/step - loss: 0.6459 - accuracy: 0.6520 - val_loss: 0.6234 - val_accuracy: 0.6805
Epoch 4/50
121/121 [==============================] - 3s 26ms/step - loss: 0.6395 - accuracy: 0.6507 - val_loss: 0.6167 - val_accuracy: 0.6784
Epoch 5/50
121/121 [==============================] - 3s 26ms/step - loss: 0.6096 - accuracy: 0.6540 - val_loss: 0.5528 - val_accuracy: 0.7210
Epoch 6/50
121/121 [==============================] - 3s 27ms/step - loss: 0.5318 - accuracy: 0.7257 - val_loss: 0.4558 - val_accuracy: 0.7697
Epoch 7/50
121/121 [==============================] - 3s 25ms/step - loss: 0.4984 - accuracy: 0.7482 - val_loss: 0.4741 - val_accuracy: 0.7490

In [57]:
data_point = np.array([[5520,80,-3.674076,	40.480649,21,0,11,9]])  # Replace ... with your data

# Scale and reshape the data point
# scaled_data_point = scaler.transform(data_point)
# reshaped_data_point = np.reshape(scaled_data_point, (1, 1, scaled_data_point.shape[1]))

# # Predict using the trained model
# prediction = model.predict(reshaped_data_point)

scaled_data_point = scaler.transform(data_point)
reshaped_data_point = np.reshape(scaled_data_point, (1,scaled_data_point.shape[1],1))

# Predict using the trained model
prediction = model.predict(reshaped_data_point)
# Convert the prediction to congestion or not
if prediction > 0.5:
    congestion = f"Congestion chances: {prediction}"
else:
    congestion = f"No Congestion {prediction}"

# Print the prediction
print("Prediction:", congestion)

C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 2s 2s/step
Prediction: Congestion chances: [[0.9856606]]


In [60]:
# pip install schedule
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import schedule
import time

# Define your data and model
# ...

# Define a function to train the model
def train_model():
    # Data preprocessing
    
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(features)
    X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

    # Create the LSTM model
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)
    print("5 min passed!")

# Define the schedule to train the model every 5 minutes
schedule.every(5).minutes.do(train_model)

# Run the scheduler indefinitely
while True:
    schedule.run_pending()
    time.sleep(1)


hihihihi
Epoch 1/50
121/121 [==============================] - 25s 98ms/step - loss: 0.6501 - accuracy: 0.6512 - val_loss: 0.6266 - val_accuracy: 0.6805
Epoch 2/50
121/121 [==============================] - 9s 74ms/step - loss: 0.6474 - accuracy: 0.6520 - val_loss: 0.6287 - val_accuracy: 0.6805
Epoch 3/50
121/121 [==============================] - 9s 74ms/step - loss: 0.6444 - accuracy: 0.6520 - val_loss: 0.6216 - val_accuracy: 0.6805
Epoch 4/50
121/121 [==============================] - 9s 75ms/step - loss: 0.6283 - accuracy: 0.6520 - val_loss: 0.5752 - val_accuracy: 0.7012
Epoch 5/50
121/121 [==============================] - 9s 74ms/step - loss: 0.5489 - accuracy: 0.7145 - val_loss: 0.5010 - val_accuracy: 0.7365
Epoch 6/50
121/121 [==============================] - 9s 76ms/step - loss: 0.4693 - accuracy: 0.7669 - val_loss: 0.4081 - val_accuracy: 0.8029
Epoch 7/50
121/121 [==============================] - 9s 75ms/step - loss: 0.4554 - accuracy: 0.7742 - val_loss: 0.3987 - val_accura

Epoch 5/50
121/121 [==============================] - 9s 77ms/step - loss: 0.6009 - accuracy: 0.6717 - val_loss: 0.5030 - val_accuracy: 0.7552
Epoch 6/50
121/121 [==============================] - 9s 77ms/step - loss: 0.5225 - accuracy: 0.7236 - val_loss: 0.5139 - val_accuracy: 0.7614
Epoch 7/50
121/121 [==============================] - 9s 78ms/step - loss: 0.4941 - accuracy: 0.7472 - val_loss: 0.4700 - val_accuracy: 0.7936
Epoch 8/50
121/121 [==============================] - 9s 78ms/step - loss: 0.4661 - accuracy: 0.7677 - val_loss: 0.4393 - val_accuracy: 0.7842
Epoch 9/50
121/121 [==============================] - 10s 79ms/step - loss: 0.4563 - accuracy: 0.7677 - val_loss: 0.4209 - val_accuracy: 0.7801
Epoch 10/50
121/121 [==============================] - 9s 77ms/step - loss: 0.4480 - accuracy: 0.7771 - val_loss: 0.4052 - val_accuracy: 0.8102
Epoch 11/50
121/121 [==============================] - 10s 83ms/step - loss: 0.4442 - accuracy: 0.7835 - val_loss: 0.4070 - val_accuracy: 0.

121/121 [==============================] - 11s 94ms/step - loss: 0.4988 - accuracy: 0.7431 - val_loss: 0.5021 - val_accuracy: 0.7479
Epoch 7/50
121/121 [==============================] - 11s 95ms/step - loss: 0.4711 - accuracy: 0.7542 - val_loss: 0.4436 - val_accuracy: 0.7718
Epoch 8/50
121/121 [==============================] - 12s 95ms/step - loss: 0.4514 - accuracy: 0.7693 - val_loss: 0.4173 - val_accuracy: 0.8071
Epoch 9/50
121/121 [==============================] - 12s 97ms/step - loss: 0.4426 - accuracy: 0.7825 - val_loss: 0.4718 - val_accuracy: 0.7614
Epoch 10/50
121/121 [==============================] - 11s 93ms/step - loss: 0.4304 - accuracy: 0.7900 - val_loss: 0.4140 - val_accuracy: 0.8112
Epoch 11/50
121/121 [==============================] - 11s 94ms/step - loss: 0.4264 - accuracy: 0.7942 - val_loss: 0.4044 - val_accuracy: 0.8019
Epoch 12/50
121/121 [==============================] - 11s 94ms/step - loss: 0.4239 - accuracy: 0.7944 - val_loss: 0.4047 - val_accuracy: 0.7967


Exception ignored in: <function UniquePtr.__del__ at 0x000001D557D9CCA0>
Traceback (most recent call last):
  File "C:\Users\Anup\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\framework\c_api_util.py", line 70, in __del__
    def __del__(self):
KeyboardInterrupt: 


 57/121 [=============>................] - ETA: 7s - loss: 0.6550 - accuracy: 0.6469

KeyboardInterrupt: 

In [3]:
import graphviz

# Create a new graph
graph = graphviz.Digraph()

# Add nodes to the graph
graph.node("A", "Development of AI-based prototype for TB detection")
graph.node("B", "Training the model using CNN and deep learning")
graph.node("C", "Digital image conversion from 2D to 3D")
graph.node("D", "Testing and refining the model")
graph.node("E", "Development of user-friendly GUI")
graph.node("F", "Report generation based on model findings")
graph.node("G", "Refinement of the model for other respiratory disorders")

# Add edges to the graph
graph.edge("A", "B")
graph.edge("B", "C")
graph.edge("B", "D")
graph.edge("D", "E")
graph.edge("D", "F")
graph.edge("G", "F")

# Render the graph to a file
graph.render("day_to_day_responsibilities_flowchart", format="png")
c

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH